<a href="https://colab.research.google.com/github/aravind-chilakamarri/CS5001-Fundamentals-of-Data-Engineering/blob/main/DE_pyspark_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be

# PySpark on Google Colab 101

In this article, we will see how we can run PySpark in a Google Colaboratory notebook. We will also perform some basic data exploratory tasks common to most data science problems. So, let’s get cracking!

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz

--2023-12-09 00:44:12--  https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272938638 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.4-bin-hadoop2.7.tgz’

spark-3.2.4-bin-had 100%[===================>] 260.29M  25.6MB/s    in 11s     

2023-12-09 00:44:23 (23.9 MB/s) - ‘spark-3.2.4-bin-hadoop2.7.tgz’ saved [272938638/272938638]



In [5]:
!tar xf spark-3.2.4-bin-hadoop2.7.tgz

In [6]:
!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
spark

In [11]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /content/sample_books.json

--2023-12-09 00:44:47--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/content/sample_books.json’

/content/sample_boo 100%[===================>]   1.53K  --.-KB/s    in 0s      

2023-12-09 00:44:48 (19.3 MB/s) - ‘/content/sample_books.json’ saved [1565/1565]



In [12]:
!wget https://gist.githubusercontent.com/jaidevd/23aef12e9bf56c618c41/raw/c05e98672b8d52fa0cb94aad80f75eb78342e5d4/books.csv

--2023-12-09 00:44:48--  https://gist.githubusercontent.com/jaidevd/23aef12e9bf56c618c41/raw/c05e98672b8d52fa0cb94aad80f75eb78342e5d4/books.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12151 (12K) [text/plain]
Saving to: ‘books.csv’

books.csv           100%[===================>]  11.87K  --.-KB/s    in 0.001s  

2023-12-09 00:44:48 (20.2 MB/s) - ‘books.csv’ saved [12151/12151]



In [13]:

# Read JSON file into dataframe
df = spark.read.json("/content/sample_books.json")


In [14]:
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [15]:
df.show(20,False)

+--------------------+-----------------+-----+-----------------------------+------------+
|author              |edition          |price|title                        |year_written|
+--------------------+-----------------+-----+-----------------------------+------------+
|Austen, Jane        |Penguin          |18.2 |Northanger Abbey             |1814        |
|Tolstoy, Leo        |Penguin          |12.7 |War and Peace                |1865        |
|Tolstoy, Leo        |Penguin          |13.5 |Anna Karenina                |1875        |
|Woolf, Virginia     |Harcourt Brace   |25.0 |Mrs. Dalloway                |1925        |
|Cunnningham, Michael|Harcourt Brace   |12.35|The Hours                    |1999        |
|Twain, Mark         |Penguin          |5.76 |Huckleberry Finn             |1865        |
|Dickens, Charles    |Random House     |5.75 |Bleak House                  |1870        |
|Twain, Mark         |Random House     |7.75 |Tom Sawyer                   |1862        |
|Woolf, Vi

In [16]:
df.count()

13

In [17]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [18]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")

In [19]:
df_filtered.select("title", "price", "year_written").show(5, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [20]:
df_filtered.count()

3

In [21]:
df_filtered.show(20, False)

+--------------------+-----------------+-----+-----------------------------+------------+
|author              |edition          |price|title                        |year_written|
+--------------------+-----------------+-----+-----------------------------+------------+
|Cunnningham, Michael|Harcourt Brace   |12.35|The Hours                    |1999        |
|Rowling, J.K.       |Harcourt Brace   |19.95|Harry Potter                 |2000        |
|Marquez             |Harper  Perennial|14.0 |One Hundred Years of Solitude|1967        |
+--------------------+-----------------+-----+-----------------------------+------------+



In [22]:
df_filtered.select("title", "year_written").filter("title LIKE '%Harry Potter%'").distinct().show(20, False)


+------------+------------+
|title       |year_written|
+------------+------------+
|Harry Potter|2000        |
+------------+------------+



In [23]:
from pyspark.sql.functions import max
# Find the costliest book

maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+



In [24]:
# Read csv file into dataframe for Project#12
df2 = spark.read.option("header", True).csv("/content/books.csv")

In [25]:
df2.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Publisher: string (nullable = true)



In [26]:
df2.show(5,truncate=False)

+------------------------+----------------+-----------------+------+-------------+
|Title                   |Author          |Genre            |Height|Publisher    |
+------------------------+----------------+-----------------+------+-------------+
|Fundamentals of Wavelets|Goswami, Jaideva|signal_processing|228   |Wiley        |
|Data Smart              |Foreman, John   |data_science     |235   |Wiley        |
|God Created the Integers|Hawking, Stephen|mathematics      |197   |Penguin      |
|Superfreakonomics       |Dubner, Stephen |economics        |179   |HarperCollins|
|Orientalism             |Said, Edward    |history          |197   |Penguin      |
+------------------------+----------------+-----------------+------+-------------+
only showing top 5 rows



In [27]:
df2.count()

211

a. Find all books published by Penguin.

In [28]:
from pyspark.sql import functions as F
penguin_books = df2.filter(df2['Publisher'] == 'Penguin')

# Show all columns for Penguin books
penguin_books.show()

+--------------------+------------------+------------+------+---------+
|               Title|            Author|       Genre|Height|Publisher|
+--------------------+------------------+------------+------+---------+
|God Created the I...|  Hawking, Stephen| mathematics|   197|  Penguin|
|         Orientalism|      Said, Edward|     history|   197|  Penguin|
|Drunkard's Walk, The| Mlodinow, Leonard|     science|   197|  Penguin|
|How to Think Like...|  Konnikova, Maria|  psychology|   240|  Penguin|
|   Age of Wrath, The|    Eraly, Abraham|     history|   238|  Penguin|
|New Machiavelli, The|      Wells, H. G.|     fiction|   180|  Penguin|
|Physics & Philosophy|Heisenberg, Werner|     science|   197|  Penguin|
|Signal and the No...|      Silver, Nate|data_science|   233|  Penguin|
|Beautiful and the...|   Deb, Siddhartha|  nonfiction|   198|  Penguin|
|       Outsider, The|     Camus, Albert|     fiction|   198|  Penguin|
| Tao of Physics, The|    Capra, Fritjof|     science|   179|  P

b. Find 10 longest book titles.

In [29]:
# Calculate the length of book titles
df2_with_title_length = df2.withColumn('Title_Length', F.length(df2['Title']))

# Order by title length and get the top 10 longest titles
top_10_longest_titles = df2_with_title_length.orderBy(F.desc('Title_Length')).limit(10)

# Show all columns for the 10 longest titles
top_10_longest_titles.show()

+--------------------+------------------+-----------------+------+--------------+------------+
|               Title|            Author|            Genre|Height|     Publisher|Title_Length|
+--------------------+------------------+-----------------+------+--------------+------------+
|Clash of Civiliza...|Huntington, Samuel|          history|   228|Simon&Schuster|          54|
|Structure & Inter...|   Sussman, Gerald| computer_science|   240|     MIT Press|          47|
|Empire of the Mug...|  Rutherford, Alex|          history|   180|          null|          45|
|Nature of Statist...|  Vapnik, Vladimir|     data_science|   230|      Springer|          42|
|Image Processing ...|       Shih, Frank|signal_processing|   241|           CRC|          42|
|Empire of the Mug...|  Rutherford, Alex|          history|   180|          null|          42|
|Empire of the Mug...|  Rutherford, Alex|          history|   180|          null|          41|
|Empire of the Mug...|  Rutherford, Alex|         

c. Find the author with the greatest number of books


In [52]:
# Group by 'Author', count the number of books per author, and find the author with the max count
author_counts = df2.groupBy('Author').count()
max_books_count = author_counts.agg({'count': 'max'}).collect()[0][0]

# Get the author(s) with the max count of books
author_with_max_books = author_counts.filter(author_counts['count'] == max_books_count).select('Author').collect()

# Filter the DataFrame to get books by the author(s) with the most books
books_by_top_author = df2.filter(df2['Author'].isin([row.Author for row in author_with_max_books]))

# Show the books by the author(s) with the most books
books_by_top_author.show()

+-----+------+-----+------+---------+
|Title|Author|Genre|Height|Publisher|
+-----+------+-----+------+---------+
+-----+------+-----+------+---------+



d. Find books included in the genre with the greatest number of books.

In [53]:
# Group by 'Genre', count the number of books per genre, and find the genre with the max count
genre_counts = df2.groupBy('Genre').count()
max_genre_count = genre_counts.agg({'count': 'max'}).collect()[0][0]

# Get the genre(s) with the max count of books
genre_with_max_books = genre_counts.filter(genre_counts['count'] == max_genre_count).select('Genre').collect()

# Filter the DataFrame to get books from the genre(s) with the most books
books_in_top_genre = df2.filter(df2['Genre'].isin([row.Genre for row in genre_with_max_books]))

# Show the books in the genre(s) with the most books
books_in_top_genre.show()

+--------------------+-------------------+-------+------+-------------+
|               Title|             Author|  Genre|Height|    Publisher|
+--------------------+-------------------+-------+------+-------------+
| Slaughterhouse Five|     Vonnegut, Kurt|fiction|   198| Random House|
|          Trial, The|       Kafka, Frank|fiction|   198| Random House|
|New Machiavelli, The|       Wells, H. G.|fiction|   180|      Penguin|
|       Outsider, The|      Camus, Albert|fiction|   198|      Penguin|
|Complete Sherlock...|Doyle, Arthur Conan|fiction|   176| Random House|
|Complete Sherlock...|Doyle, Arthur Conan|fiction|   176| Random House|
|Pillars of the Ea...|       Follett, Ken|fiction|   176| Random House|
| Farewell to Arms, A|  Hemingway, Ernest|fiction|   179|         Rupa|
|        Veteran, The| Forsyth, Frederick|fiction|   177|   Transworld|
|   False Impressions|    Archer, Jeffery|fiction|   177|          Pan|
|       Jurassic Park|  Crichton, Michael|fiction|   174| Random

e. Find all records with at least one of five columns that has null in it.

In [59]:
from pyspark.sql.functions import col

# Find all records with at least one null value in the specified columns
records_with_null = df2.filter(col("height").isNull() | col("title").isNull() | col("author").isNull() | col("publisher").isNull() | col("genre").isNull())
records_with_null.show()

+--------------------+--------------------+----------+------+-------------+
|               Title|              Author|     Genre|Height|    Publisher|
+--------------------+--------------------+----------+------+-------------+
|      Beyond Degrees|                null|nonfiction|   222|HarperCollins|
|World's Greatest ...|                null|   history|   210|         null|
|Case of the Lame ...|Gardner, Earle St...|   fiction|   179|         null|
|World's Greatest ...|                null|   fiction|   217|        Jaico|
|    Talking Straight|         Iacoca, Lee|nonfiction|   175|         null|
|Phantom of Manhat...|  Forsyth, Frederick|   fiction|   180|         null|
|Selected Short St...|                null|   fiction|   215|        Jaico|
| Karl Marx Biography|                null|nonfiction|   162|         null|
|         Half A Life|        Naipaul, V S|   fiction|   196|         null|
|Discovery of Indi...|   Nehru, Jawaharlal|   history|   230|         null|
|           